# Dotplots of aligned Fxy introns

Recall that FXY in *Mus musculus* (Mmus) has been translocated to straddle the pseudo-autosomal boundary and that it is conjectured that this boundary actually lies within intron 3. In particular, introns 1 and 2 are X-linked, while introns 4-9 are in the pseudo-autosomal region (PAR). To evaluate the quality of FXY intron alignments, we do dot plots between the two mouse in-group species – Mmus and *Mus spretus* (Mspre) – and the rat (Rnor). The cogent3 dotplots display exhaustive sequence matching (12/20 bases in a 20bp window are identical) and, for aligned sequences, the path identified by the alignment algorithm. Regions where there is no obvious diagonal from the sequence comparison, or the diagonal is not overlaid by the alignment path, are indicative of low quality alignments. Note that in some cases, the poor alignmment quality is due to the presence of ambiguous sequence states in Mspre.

We identified the first 4000bp of the Intron 2 as the best quality alignment for an X-linked alignment segment. We further identified Introns 4-6 as high quality PAR alignments. These selected alignment regions were selected for downstream analyses.

In [1]:
from cogent3 import load_aligned_seqs
from plotly.subplots import make_subplots
from project_paths import SUPP_FIG_DIR, DATA_DIR


def do_dotplot(aln, name1, name2, name3, path_stem, window=20, threshold=12, k=7):
    title = path_stem.stem.title().replace("_", " ")
    dp1 = aln.dotplot(
        name1=name1, name2=name2, window=window, threshold=threshold, k=k
    )
    dp2 = aln.dotplot(
        name1=name1, name2=name3, window=window, threshold=threshold, k=k
    )
    subplot = make_subplots(rows=1, cols=2, shared_yaxes=True, horizontal_spacing=0.01)
    f1 = dp1.plotly_figure
    for trace in f1.data:
        trace.showlegend = False
        subplot.add_trace(trace, row=1, col=1)
    f2 = dp2.plotly_figure
    for trace in f2.data:
        subplot.add_trace(trace, row=1, col=2)
    subplot.update_layout(title_text=title)
    subplot.update_yaxes(title_text=name1, row=1, col=1)
    subplot.update_xaxes(title_text=name2, row=1, col=1)
    subplot.update_xaxes(title_text=name3, row=1, col=2)
    return subplot
                          


ROOTDIR = DATA_DIR / "raw/fxy/introns"
fxy_dir = ROOTDIR / "mmu_msp_rno-aligned"
selected_dir = ROOTDIR / "selected"
selected_dir.mkdir(exist_ok=True)
dims = dict(height=200, width=1000)
paths = list(fxy_dir.glob("*.fa"))
paths = {int(p.stem.split("_")[1]): p for p in paths}

# By intron

In [2]:
for i in range(1, len(paths) + 1):
    p = paths[i]
    out_path = SUPP_FIG_DIR / f"fxy-dotplot-{p.stem.replace('_', '-')}.png"
    aln = load_aligned_seqs(p, moltype="dna")
    dp = do_dotplot(aln, "Mmus", "Mspr", "Rnor", p)
    dp.update_layout(title_y=0.98, margin=dict(l=20, r=12, t=30, b=20), **dims)
    # double the DPI over default for better resolution
    dp.write_image(str(out_path), scale=2)
    # dp.show()